In [106]:
# visualization part

import jgraph
import neo

def jgraph_vis(pname):
    query = """
            MATCH (m)-[:HasPrerequisite]->(n) 
            WHERE (m)-[:UnderProgram]->({name:'"""+pname+"""'}) 
            and (n)-[:UnderProgram]->({name:'"""+pname+"""'}) 
            RETURN m.id, n.id
            """
    adjPair2 = graph.run(query)

    adjList = []
    for res in adjPair2:
        adjList.append(res)
    print("-----play with 3D Directed Acyclic Graph----")
    jgraph.draw(adjList)


def visualize(pname):
    query = """
        MATCH (n:Program{name:'"""+pname+"""'})<-[r]-(m)
        RETURN n AS source_node,
               id(n) AS source_id,
               r,
               m AS target_node,
               id(m) AS target_id
        UNION MATCH (p:Program{name:'"""+pname+"""'})-[rr]-(n)-[r]-(m:MOOC)
        RETURN n AS source_node,
               id(n) AS source_id,
               r,
               m AS target_node,
               id(m) AS target_id
            """
    d = graph.run(query)
    print("       ----------------Complete Subject Graph Under " + pname + " Program------------" )
    display(HTML(neo.draw(d,{})))
    


def visualize_pre_DAG(pname):
    query2 = """
            MATCH (m:Course)-[r]->(n) 
            WHERE (m)-[:UnderProgram]->({name: '"""+pname+"""'}) 
            and (n)-[:UnderProgram]->({name: '"""+pname+"""'}) 
            RETURN m As source_node, m.id As source_id, r, 
                   n As target_node, n.id As target_id

              """

    d2 = graph.run(query2)
    print("      -------------Required Course Enrollment Subgraph Under " + pname + " Program------------" )
    display(HTML(neo.draw(d2,{})))

# print("--------------------DISPLAY KNOWLEDGE GRAPH-------------------------")
# visualize("Accounting (BS)")
# pname = "Accounting (BS)"
# visualize_pre_DAG(pname)
# jgraph_vis(pname)


In [107]:
## topological listing part

import json, jgraph
import pandas as pd
from py2neo import Graph
from collections import defaultdict


class Prerequisite:
    def __init__(self, courselist):
        self.graph = defaultdict(list) 
        self.vertices = courselist
 
    def addEdge(self, u, v):
        self.graph[u].append(v)
 
    def dfs(self, v, visit, order):
        visit[v] = True
        for u in self.graph[v]:
            if visit[u] == False:
                self.dfs(u, visit, order)
        order.insert(0, v)
 
    def tps(self):
        visit = {v: False for v in self.vertices}
        order =[]
        for v in self.vertices:
            if visit[v] == False:
                self.dfs(v, visit, order)
        return order
    
    
def refernce_display(clist):
    content = '''
        <table style="width:100%" bordercolor='lightgreen'>
          <tr>
            <th style="text-align:left"; width="10%;">Code</th>
            <th style="text-align:left"; width="30%;">Name</th> 
            <th style="text-align:left"; width="20%;">Image</th>
            <th style="text-align:left"; width="40%;">MOOC</th>
          </tr>
          '''
    for c in clist:
        content += '''
            <tr>
                <td style="text-align:left";>''' + c['code'] + '''</td>
                <td style="text-align:left";>''' + c['name'] + '''</td> 
                <td><img src=''' + c['image'] + ''' height="120px"; display: inline-block;></td>
                <td style="text-align:left";> <a href="''' + c['url'] + '''" target="_blank">''' + c['mooc'] + '''</td></tr>'''
    #return content
    display(HTML(content))
    
    
def display_list(order):
    nodes, relat = [], []
    course, relation = list(), list()

    for id in order:
        dic = {}
        dic["code"], dic["mooc"], dic["url"], dic["image"] = id, "", "", ""
        node = graph.data("Match (n:Course{id:'" + id + "'}) RETURN n")
        coursename = graph.data("Match (n:Course{id:'" + id + "'}) RETURN n.name")[0]
        coursedesc = graph.data("Match (n:Course{id:'" + id + "'}) RETURN n.description")[0]
        mooc = graph.data("MATCH p = (n:Course{id:'"+ id + "'})-[r:SimilarTo]->(m:MOOC) RETURN m.name limit 1")
        image = graph.data("MATCH p = (n:Course{id:'"+ id + "'})-[r:SimilarTo]->(m:MOOC) RETURN m.image limit 1")
        url = graph.data("MATCH p = (n:Course{id:'"+ id + "'})-[r:SimilarTo]->(m:MOOC) RETURN m.url limit 1")
        dic["name"] = coursename["n.name"]
        if len(mooc) != 0:
            rlist = [id, coursename["n.name"], mooc[0]["m.name"], image[0]["m.image"], url[0]["m.url"]]
            relation.append(rlist)
            dic["mooc"], dic["image"], dic["url"] = rlist[2], rlist[3], rlist[4]
        relat.append(dic)
        nodes += node
        course.append([id, coursename["n.name"], coursedesc["n.description"]])
    course_df = pd.DataFrame(course)
    relation_df = pd.DataFrame(relation)
    refernce_display(relat)
    j = json.dumps(relat)

def whole_id(pname):
    query2 = """
            MATCH (m)
            WHERE (m)-[:UnderProgram]->({name:'"""+ pname +"""'}) 
            RETURN m.id as id
            """
    data = graph.run(query2)
    id_list = []
    for id in data:
        id_list.append(id["id"])
    return id_list


def topo_Sort(pname):
    graph = Graph("http://localhost:7474/db/data/")
    query = """
            MATCH (m)-[:HasPrerequisite]->(n) 
            WHERE (m)-[:UnderProgram]->({name:'"""+ pname +"""'}) 
            and (n)-[:UnderProgram]->({name:'"""+ pname +"""'}) 
            RETURN m.id, n.id 
            """
    adjPair = graph.data(query)

    courselist = []
    for i in adjPair:
        courselist += [i["m.id"], i["n.id"]]
    courselist = list(set(courselist))

    g = Prerequisite(courselist)
    for edge in adjPair:
        g.addEdge(edge["m.id"], edge["n.id"])
    toposort = g.tps()
    order = toposort[::-1]
    display(HTML("<h6 style=' font-size: 15px; text-align:center;'>Study Path I: Program-required Enrolling Order with Recommended MOOCs</h6>"))
    display_list(order)

    display(HTML("<h6 style=' font-size: 15px; text-align:center;'>Study Path II: Self-planned Enrolling Order with Recommended MOOCs</h6>"))
    other = [i for i in whole_id(pname) if i not in order]
    display_list(other)

In [108]:
# Fuzzy Search Part

from py2neo import Graph,NodeSelector
import neo
from ipywidgets import *
from IPython.display import display, HTML
neo.init_notebook_mode() 
graph = Graph("http://localhost:7474/db/data/")

searchbox = widgets.Text(
        placeholder='Program Name',
        description='Search:',
        disabled=False)
    
def program_result(plist):
    items_layout = Layout(
        flex='1 1 auto',
        width='auto')     # override the default width of the button to 'auto' to let the button grow

    box_layout = Layout(
        display='flex',
        flex_flow='column',
        align_items='stretch',
        width='60%')
    items = [ToggleButton(description=w, layout=items_layout) for w in plist]
    return Box(children=items, layout=box_layout)


def search_program(keyword):
    query = "MATCH (p:Program) WHERE p.name =~ '(?i).*"+keyword+".*' RETURN p LIMIT 10"
    data = graph.data(query)
    return [p['p']['name'] for p in data]

def value_changed(change):
    res = search_program(change.new)
    box = program_result(res)
    display(box)
    value_changed.box = box.children

searchbox.observe(value_changed, 'value')

display(searchbox)

from IPython.core.display import HTML
def on_button_clicked(b):
    for box in value_changed.box:
        if box.value == True:
            selected = box.description
            query = """
                    MATCH (m:Program {name:'"""+ selected +"""'}) 
                    RETURN m.url
                    """
            link = (graph.data(query)[0]["m.url"])
#             print(link)
            visualize(selected)
            
            display(HTML("<div style='color:lightblue; font-size: 14px; text-align:center;'><a href=" +link+ "> Visit Program Homepage !</a></div>"))
            visualize_pre_DAG(selected)
            topo_Sort(selected)
            
#   demo_key = ["da", "accounting"]
button = widgets.Button(description="See Detail !")
button.on_click(on_button_clicked)
display(button)

<IPython.core.display.Javascript object>

Text(value='', description='Search:', placeholder='Program Name')

Button(description='See Detail !', style=ButtonStyle())

In [10]:
from py2neo import Graph
from collections import defaultdict

graph = Graph("http://localhost:7474/db/data/")
adjPair = graph.data("MATCH (m)-[:HasPrerequisite]->(n) \
        WHERE (m)-[:UnderProgram]->({name: 'Accounting (BS)'}) \
        and (n)-[:UnderProgram]->({name: 'Accounting (BS)'}) \
        RETURN m.id, n.id")

class Prerequisite:
    def __init__(self, courselist):
        self.graph = defaultdict(list) 
        self.vertices = courselist
 
    def addEdge(self, u, v):
        self.graph[u].append(v)
 
    def dfs(self, v, visit, order):
        visit[v] = True
        for u in self.graph[v]:
            if visit[u] == False:
                self.dfs(u, visit, order)
        order.insert(0, v)
 
    def tps(self):
        visit = {v: False for v in self.vertices}
        order =[]
        for v in self.vertices:
            if visit[v] == False:
                self.dfs(v, visit, order)
        return order
courselist = []
for i in adjPair:
    courselist += [i["m.id"], i["n.id"]]
courselist = list(set(courselist))

g = Prerequisite(courselist)
for edge in adjPair:
    g.addEdge(edge["m.id"], edge["n.id"])
# MATCH (n:Course{id:"BUAD 280"}) RETURN n

toposort = g.tps()
print(toposort[::-1])
nodes = []
course = list()
relation = list()
relat = []
import pandas as pd
for id in toposort[::-1]:
    dic = {}
    node = graph.data("Match (n:Course{id:'" + id + "'}) RETURN n")
    coursename = graph.data("Match (n:Course{id:'" + id + "'}) RETURN n.name")[0]
    coursedesc = graph.data("Match (n:Course{id:'" + id + "'}) RETURN n.description")[0]
    mooc = graph.data("MATCH p = (n:Course{id:'"+ id + "'})-[r:SimilarTo]->(m:MOOC) RETURN m.name limit 1")
    image = graph.data("MATCH p = (n:Course{id:'"+ id + "'})-[r:SimilarTo]->(m:MOOC) RETURN m.image limit 1")
    url = graph.data("MATCH p = (n:Course{id:'"+ id + "'})-[r:SimilarTo]->(m:MOOC) RETURN m.url limit 1")
    print(mooc)
    dic["code"] = id
    dic["name"] = coursename["n.name"]
    dic["mooc"] = ""
    dic["url"] = ""
    dic["image"] = ""
    if len(mooc) != 0:
        rlist = [id, coursename["n.name"], mooc[0]["m.name"], image[0]["m.image"], url[0]["m.url"]]
        relation.append(rlist)
        dic["mooc"], dic["image"], dic["url"] = rlist[2], rlist[3], rlist[4]
    relat.append(dic)
    nodes += node
    course.append([id, coursename["n.name"], coursedesc["n.description"]])
    
course_df = pd.DataFrame(course)
relation_df = pd.DataFrame(relation)
relat
import json
j = json.dumps(relat)

# nodes
# print(nodes)


course_list = [{'code': 'BUAD 280',
  'image': 'https://d3njjcbhbojbot.cloudfront.net/api/utilities/v1/imageproxy/https://coursera-course-photos.s3.amazonaws.com/f2/fdfaf0f9a511e6a5f4effa0e2c4d64/investigaci_n-de-mercados-y-comportamiento.jpg?auto=format%2Ccompress&dpr=1&w=100&h=100&fit=fill&bg=FFF',
  'mooc': 'Accounting: Principles of Financial Accounting',
  'name': 'BUAD 280 Introduction to Financial Accounting',
  'url': 'http://www.coursera.org/specializations/foundations-management'},
 {'code': 'MATH 118',
  'image': '',
  'mooc': '',
  'name': 'MATH 118gx Fundamental Principles of Calculus',
  'url': ''},
 {'code': 'MATH 125',
  'image': 'https://d3njjcbhbojbot.cloudfront.net/api/utilities/v1/imageproxy/https://coursera-course-photos.s3.amazonaws.com/05/cda840977511e5aa161903ab17f92e/CSV-logo-chapter-5-copy.jpg?auto=format%2Ccompress&dpr=1&w=100&h=100&fit=fill&bg=FFF',
  'mooc': 'Single Variable Calculus',
  'name': 'MATH 125g Calculus I',
  'url': 'http://www.coursera.org/learn/discrete-calculus'},
 {'code': 'ECON 351',
  'image': 'https://d3njjcbhbojbot.cloudfront.net/api/utilities/v1/imageproxy/https://coursera-course-photos.s3.amazonaws.com/06/ab77207de611e78b8cbf6b7d2487ac/image-1.jpg?auto=format%2Ccompress&dpr=1&w=100&h=100&fit=fill&bg=FFF',
  'mooc': 'Strategic Business Management - Microeconomics',
  'name': 'ECON 351x Microeconomics for Business',
  'url': 'http://www.coursera.org/learn/parprog1'},
 {'code': 'BUAD 306',
  'image': 'https://d3njjcbhbojbot.cloudfront.net/api/utilities/v1/imageproxy/https://coursera-course-photos.s3.amazonaws.com/4c/868cb0658911e6892f017139b2b060/coursera-ma-thumbnail.jpg?auto=format%2Ccompress&dpr=1&w=100&h=100&fit=fill&bg=FFF',
  'mooc': 'Business English: Finance and Economics',
  'name': 'BUAD 306 Business Finance',
  'url': 'http://www.coursera.org/specializations/english-for-business'},
 {'code': 'BUAD 281',
  'image': 'https://d3njjcbhbojbot.cloudfront.net/api/utilities/v1/imageproxy/https://d396qusza40orc.cloudfront.net/phoenixassets/learn-finance/BNY_Specialization_Banner_logo.jpg?auto=format%2Ccompress&dpr=1&w=100&h=100&fit=fill&bg=FFF',
  'mooc': 'Managerial Accounting: Cost Behaviors, Systems, and Analysis',
  'name': 'BUAD 281 Introduction to Managerial Accounting',
  'url': 'http://www.coursera.org/specializations/value-chain-management'},
 {'code': 'ACCT 370',
  'image': '',
  'mooc': '',
  'name': 'ACCT 370 External Financial Reporting Issues',
  'url': ''},
 {'code': 'ACCT 371',
  'image': 'https://d3njjcbhbojbot.cloudfront.net/api/utilities/v1/imageproxy/https://d15cw65ipctsrr.cloudfront.net/dd/7ae3d0bd8411e6834f83bd22b5a2e6/PositivePsychology_MOOC_Icons_Specialization.jpg?auto=format%2Ccompress&dpr=1&w=100&h=100&fit=fill&bg=FFF',
  'mooc': 'Accounting and Finance for IT professionals',
  'name': 'ACCT 371 Introduction to Accounting Systems',
  'url': 'http://www.coursera.org/specializations/business-technology-managment'},
 {'code': 'ACCT 373',
  'image': '',
  'mooc': '',
  'name': 'ACCT 373 Introduction to Auditing and Assurance Services',
  'url': ''},
 {'code': 'ACCT 377',
  'image': '',
  'mooc': '',
  'name': 'ACCT 377 Valuation for Financial Statement Purposes',
  'url': ''},
 {'code': 'ACCT 416',
  'image': 'https://d3njjcbhbojbot.cloudfront.net/api/utilities/v1/imageproxy/https://coursera-course-photos.s3.amazonaws.com/3e/b505c0ee7611e4890b43504b150d21/online_learning_slide_vAccounting.jpg?auto=format%2Ccompress&dpr=1&w=100&h=100&fit=fill&bg=FFF',
  'mooc': 'More Introduction to Financial Accounting',
  'name': 'ACCT 416 Financial Reporting and Analysis',
  'url': 'http://www.coursera.org/learn/wharton-financial-accounting'},
 {'code': 'ACCT 462',
  'image': '',
  'mooc': '',
  'name': 'ACCT 462 Detecting Fraudulent Financial Reporting',
  'url': ''},
 {'code': 'ACCT 476',
  'image': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTWVTAsQKuMSTnREno6zNLCslVti6JN7G9RHpDFXaikAiF36rquQakV1igF',
  'mooc': 'Measurement and data ',
  'name': 'ACCT 476 Performance Measurement Issues',
  'url': 'https://www.khanacademy.org/math/cc-2nd-grade-math/cc-2nd-measurement-data'},
 {'code': 'BUAD 302',
  'image': '',
  'mooc': '',
  'name': 'BUAD 302T Communication Strategy in Accounting',
  'url': ''},
 {'code': 'BUAD 307',
  'image': 'https://s3-us-west-1.amazonaws.com/udacity-content/course-images/cx11-e152d95.jpg',
  'mooc': 'Digital Marketing Fundamentals',
  'name': 'BUAD 307 Marketing Fundamentals',
  'url': 'https://www.udacity.com/course/digital-marketing-fundamentals--cx11'},
 {'code': 'ACCT 477',
  'image': '',
  'mooc': '',
  'name': 'ACCT 477 Intermediate Fair Value Issues in Accounting',
  'url': ''},
 {'code': 'ACCT 478',
  'image': 'https://d3njjcbhbojbot.cloudfront.net/api/utilities/v1/imageproxy/https://coursera-course-photos.s3.amazonaws.com/3e/b505c0ee7611e4890b43504b150d21/online_learning_slide_vAccounting.jpg?auto=format%2Ccompress&dpr=1&w=100&h=100&fit=fill&bg=FFF',
  'mooc': 'More Introduction to Financial Accounting',
  'name': 'ACCT 478 Accounting Systems Design',
  'url': 'http://www.coursera.org/learn/wharton-financial-accounting'},
 {'code': 'ACCT 473',
  'image': '',
  'mooc': '',
  'name': 'ACCT 473 Financial Statement Auditing',
  'url': ''},
 {'code': 'ACCT 475',
  'image': 'https://lh3.googleusercontent.com/pjV2kaEQlQuzZSTR8YwIYyDieRxdfH6i0QfGzSCI1iTawtnhYJmbn2RC0rXdsnSGf6FaoIuLHE11NyFL=s0#w=1280&h=720=s276#w=1724&h=1060',
  'mooc': 'Cyber-Physical Systems Security',
  'name': 'ACCT 475 Systems Security and Audit',
  'url': 'https://www.udacity.com/course/cyber-physical-systems-security--ud279'},
 {'code': 'ACCT 463',
  'image': '',
  'mooc': '',
  'name': 'ACCT 463 Internal Audit',
  'url': ''},
 {'code': 'BUAD 304',
  'image': 'https://d3njjcbhbojbot.cloudfront.net/api/utilities/v1/imageproxy/https://coursera-course-photos.s3.amazonaws.com/62/58d990f18cd48083251c3e57fe9e7e/Untitled-1.jpg?auto=format%2Ccompress&dpr=1&w=100&h=100&fit=fill&bg=FFF',
  'mooc': 'International Leadership and Organizational Behavior',
  'name': 'BUAD 304 Organizational Behavior and Leadership',
  'url': 'http://www.coursera.org/learn/academic-discussion-english'},
 {'code': 'BUAD 497',
  'image': 'https://d3njjcbhbojbot.cloudfront.net/api/utilities/v1/imageproxy/https://coursera-course-photos.s3.amazonaws.com/d3/d90f107de511e7b945f75a7283d500/image-2.jpg?auto=format%2Ccompress&dpr=1&w=100&h=100&fit=fill&bg=FFF',
  'mooc': 'Strategic Business Management - Macroeconomics',
  'name': 'BUAD 497 Strategic Management',
  'url': 'http://www.coursera.org/learn/trigonometry'},
 {'code': 'ACCT 479',
  'image': 'https://d3njjcbhbojbot.cloudfront.net/api/utilities/v1/imageproxy/https://coursera-course-photos.s3.amazonaws.com/ef/485800fcfd11e6b4cc3359e41f40e9/Pricing-Strategy.jpg?auto=format%2Ccompress&dpr=1&w=100&h=100&fit=fill&bg=FFF',
  'mooc': 'Accounting Analytics',
  'name': 'ACCT 479 Accounting Systems Development',
  'url': 'http://www.coursera.org/specializations/business-analytics'},
 {'code': 'ACCT 374',
  'image': '',
  'mooc': '',
  'name': 'ACCT 374 Introduction to Tax Issues',
  'url': ''},
 {'code': 'ACCT 474',
  'image': '',
  'mooc': '',
  'name': 'ACCT 474 Tax Issues for Business',
  'url': ''},
 {'code': 'ACCT 470',
  'image': '',
  'mooc': '',
  'name': 'ACCT 470 Advanced External Financial Reporting Issues',
  'url': ''},
 {'code': 'ACCT 471',
  'image': 'https://d3njjcbhbojbot.cloudfront.net/api/utilities/v1/imageproxy/https://coursera-course-photos.s3.amazonaws.com/e7/871c10757e11e7817cc9840feed7fa/enigma_rotors_with_alphabet_rings_cropped.jpg?auto=format%2Ccompress&dpr=1&w=100&h=100&fit=fill&bg=FFF',
  'mooc': 'Accounting Analytics',
  'name': 'ACCT 471 Accounting Information Systems',
  'url': 'http://www.coursera.org/specializations/business-analytics'},
 {'code': 'ACCT 372',
  'image': '',
  'mooc': '',
  'name': 'ACCT 372 Internal Reporting Issues',
  'url': ''},
 {'code': 'ACCT 472',
  'image': 'https://d3njjcbhbojbot.cloudfront.net/api/utilities/v1/imageproxy/https://d396qusza40orc.cloudfront.net/phoenixassets/learn-finance/BNY_Specialization_Banner_logo.jpg?auto=format%2Ccompress&dpr=1&w=100&h=100&fit=fill&bg=FFF',
  'mooc': 'Managerial Accounting: Cost Behaviors, Systems, and Analysis',
  'name': 'ACCT 472 Managerial Accounting',
  'url': 'http://www.coursera.org/specializations/value-chain-management'},
 {'code': 'ACCT 430',
  'image': 'https://d3njjcbhbojbot.cloudfront.net/api/utilities/v1/imageproxy/https://coursera-course-photos.s3.amazonaws.com/ef/485800fcfd11e6b4cc3359e41f40e9/Pricing-Strategy.jpg?auto=format%2Ccompress&dpr=1&w=100&h=100&fit=fill&bg=FFF',
  'mooc': 'Accounting Analytics',
  'name': 'ACCT 430 Accounting Ethics',
  'url': 'http://www.coursera.org/specializations/business-analytics'}]
course_df
refernce_display(relat)

['BUAD 280', 'BUAD 281', 'ACCT 371', 'ACCT 478', 'ACCT 476', 'ACCT 370', 'ACCT 377', 'ACCT 471', 'ACCT 475', 'MATH 125', 'BUAD 302', 'ACCT 372', 'MATH 118', 'ECON 351', 'BUAD 306', 'ACCT 374', 'ACCT 373', 'ACCT 473', 'ACCT 463', 'ACCT 472', 'BUAD 304', 'BUAD 307', 'BUAD 497', 'ACCT 462', 'ACCT 470', 'ACCT 477', 'ACCT 416', 'ACCT 474', 'ACCT 430', 'ACCT 479']
[{'m.name': 'Accounting: Principles of Financial Accounting'}]
[{'m.name': 'Managerial Accounting: Cost Behaviors, Systems, and Analysis'}]
[{'m.name': 'Accounting and Finance for IT professionals'}]
[{'m.name': 'More Introduction to Financial Accounting'}]
[{'m.name': 'Measurement and data '}]
[]
[]
[{'m.name': 'Accounting Analytics'}]
[{'m.name': 'Cyber-Physical Systems Security'}]
[{'m.name': 'Single Variable Calculus'}]
[]
[]
[]
[{'m.name': 'Strategic Business Management - Microeconomics'}]
[{'m.name': 'Business English: Finance and Economics'}]
[]
[]
[]
[]
[{'m.name': 'Managerial Accounting: Cost Behaviors, Systems, and Analys

NameError: name 'refernce_display' is not defined

In [11]:
refernce_display(course_list)

NameError: name 'refernce_display' is not defined

['BUAD 280', 'BUAD 281', 'BUAD 302', 'BUAD 304', 'BUAD 306', 'BUAD 307', 'BUAD 310', 'BUAD 311', 'BUAD 497', 'ECON 351', 'ECON 352', 'MATH 118', 'MATH 125', 'ACCT 370', 'ACCT 371', 'ACCT 372', 'ACCT 373', 'ACCT 374', 'ACCT 377', 'ACCT 430', 'ACCT 470', 'ACCT 473', 'ACCT 474', 'ACCT 416', 'ACCT 462', 'ACCT 463', 'ACCT 471', 'ACCT 472', 'ACCT 475', 'ACCT 476', 'ACCT 477', 'ACCT 478', 'ACCT 479', 'ACCT 493']


In [20]:
def visualize(pname):
    query = """
        MATCH (n:Program{name:'"""+pname+"""'})<-[r]-(m)
        RETURN n AS source_node,
               id(n) AS source_id,
               r,
               m AS target_node,
               id(m) AS target_id
        UNION MATCH (p:Program{name:'"""+pname+"""'})-[rr]-(n)-[r]-(m:MOOC)
        RETURN n AS source_node,
               id(n) AS source_id,
               r,
               m AS target_node,
               id(m) AS target_id
            """
    d = graph.run(query)
    display(HTML(neo.draw(d,{})))

In [21]:
def search_program(keyword):
    query = "MATCH (p:Program) WHERE p.name =~ '(?i).*"+keyword+".*' RETURN p LIMIT 10"
    data = graph.data(query)
    return [p['p']['name'] for p in data]

In [46]:
# visualization part

import jgraph
import neo

def jgraph_vis(pname):
    query = """
            MATCH (m)-[:HasPrerequisite]->(n) \
            WHERE (m)-[:UnderProgram]->({name:'"""+pname+"""'}) \
            and (n)-[:UnderProgram]->({name:'"""+pname+"""'}) \
            RETURN m.id, n.id
            """
    adjPair2 = graph.run(query)

    adjList = []
    for res in adjPair2:
        adjList.append(res)
    print("-----play with 3D Directed Acyclic Graph----")
    jgraph.draw(adjList)


def visualize(pname):
    query = """
        MATCH (n:Program{name:'"""+pname+"""'})<-[r]-(m)
        RETURN n AS source_node,
               id(n) AS source_id,
               r,
               m AS target_node,
               id(m) AS target_id
        UNION MATCH (p:Program{name:'"""+pname+"""'})-[rr]-(n)-[r]-(m:MOOC)
        RETURN n AS source_node,
               id(n) AS source_id,
               r,
               m AS target_node,
               id(m) AS target_id
            """
    d = graph.run(query)
    print("----------------Subject Graph Under " + pname + "------------" )
    display(HTML(neo.draw(d,{})))
    


def visualize_pre_DAG(pname):
    query2 = """
            MATCH (m:Course)-[r]->(n) \
            WHERE (m)-[:UnderProgram]->({name: '"""+pname+"""'}) \
            and (n)-[:UnderProgram]->({name: '"""+pname+"""'}) \
            RETURN m As source_node, m.id As source_id, r, 
                   n As target_node, n.id As target_id

              """

    d2 = graph.run(query2)
    print("-------------Course Enrollment Graph Under " + pname + "------------\n\n" )
    display(HTML(neo.draw(d2,{})))

# print("--------------------DISPLAY KNOWLEDGE GRAPH-------------------------")
# visualize("Accounting (BS)")
pname = "Accounting (BS)"
# visualize_pre_DAG(pname)
# jgraph_vis(pname)


In [65]:
import json, jgraph
import pandas as pd
from py2neo import Graph
from collections import defaultdict


class Prerequisite:
    def __init__(self, courselist):
        self.graph = defaultdict(list) 
        self.vertices = courselist
 
    def addEdge(self, u, v):
        self.graph[u].append(v)
 
    def dfs(self, v, visit, order):
        visit[v] = True
        for u in self.graph[v]:
            if visit[u] == False:
                self.dfs(u, visit, order)
        order.insert(0, v)
 
    def tps(self):
        visit = {v: False for v in self.vertices}
        order =[]
        for v in self.vertices:
            if visit[v] == False:
                self.dfs(v, visit, order)
        return order
    
    
def refernce_display(clist):
    content = '''
        <table style="width:100%" bordercolor='lightgreen'>
          <tr>
            <th style="text-align:left"; width="10%;">Code</th>
            <th style="text-align:left"; width="30%;">Name</th> 
            <th style="text-align:left"; width="20%;">Image</th>
            <th style="text-align:left"; width="40%;">MOOC</th>
          </tr>
          '''
    for c in clist:
        content += '''
            <tr>
                <td style="text-align:left";>''' + c['code'] + '''</td>
                <td style="text-align:left";>''' + c['name'] + '''</td> 
                <td><img src=''' + c['image'] + ''' height="120px"; display: inline-block;></td>
                <td style="text-align:left";> <a href="''' + c['url'] + '''" target="_blank">''' + c['mooc'] + '''</td></tr>'''
    #return content
    display(HTML(content))
    
    
def display_list(order):
    nodes, relat = [], []
    course, relation = list(), list()

    for id in order:
        dic = {}
        dic["code"], dic["mooc"], dic["url"], dic["image"] = id, "", "", ""
        node = graph.data("Match (n:Course{id:'" + id + "'}) RETURN n")
        coursename = graph.data("Match (n:Course{id:'" + id + "'}) RETURN n.name")[0]
        coursedesc = graph.data("Match (n:Course{id:'" + id + "'}) RETURN n.description")[0]
        mooc = graph.data("MATCH p = (n:Course{id:'"+ id + "'})-[r:SimilarTo]->(m:MOOC) RETURN m.name limit 1")
        image = graph.data("MATCH p = (n:Course{id:'"+ id + "'})-[r:SimilarTo]->(m:MOOC) RETURN m.image limit 1")
        url = graph.data("MATCH p = (n:Course{id:'"+ id + "'})-[r:SimilarTo]->(m:MOOC) RETURN m.url limit 1")
        dic["name"] = coursename["n.name"]
        if len(mooc) != 0:
            rlist = [id, coursename["n.name"], mooc[0]["m.name"], image[0]["m.image"], url[0]["m.url"]]
            relation.append(rlist)
            dic["mooc"], dic["image"], dic["url"] = rlist[2], rlist[3], rlist[4]
        relat.append(dic)
        nodes += node
        course.append([id, coursename["n.name"], coursedesc["n.description"]])
    course_df = pd.DataFrame(course)
    relation_df = pd.DataFrame(relation)
    print("    ----------Study Path: School Course with Recommended MOOC----------")
    refernce_display(relat)
    j = json.dumps(relat)
    print("       ----------Course Enrollment Table----------")
    return course_df


def topo_Sort(program_name):
    graph = Graph("http://localhost:7474/db/data/")
    query = """
            MATCH (m)-[:HasPrerequisite]->(n) 
            WHERE (m)-[:UnderProgram]->({name:'"""+pname+"""'}) 
            and (n)-[:UnderProgram]->({name:'"""+pname+"""'}) 
            RETURN m.id, n.id
            """
    adjPair = graph.data(query)

    courselist = []
    for i in adjPair:
        courselist += [i["m.id"], i["n.id"]]
    courselist = list(set(courselist))

    g = Prerequisite(courselist)
    for edge in adjPair:
        g.addEdge(edge["m.id"], edge["n.id"])
    # MATCH (n:Course{id:"BUAD 280"}) RETURN n
    toposort = g.tps()
    return display_list(toposort[::-1])




    ----------Study Path: School Course with Recommended MOOC----------


Code,Name,Image,MOOC
BUAD 280,BUAD 280 Introduction to Financial Accounting,,Accounting: Principles of Financial Accounting
BUAD 281,BUAD 281 Introduction to Managerial Accounting,,"Managerial Accounting: Cost Behaviors, Systems, and Analysis"
ACCT 371,ACCT 371 Introduction to Accounting Systems,,Accounting and Finance for IT professionals
ACCT 478,ACCT 478 Accounting Systems Design,,More Introduction to Financial Accounting
ACCT 476,ACCT 476 Performance Measurement Issues,,Measurement and data
BUAD 302,BUAD 302T Communication Strategy in Accounting,,
BUAD 304,BUAD 304 Organizational Behavior and Leadership,,International Leadership and Organizational Behavior
MATH 118,MATH 118gx Fundamental Principles of Calculus,,
MATH 125,MATH 125g Calculus I,,Single Variable Calculus
ECON 351,ECON 351x Microeconomics for Business,,Strategic Business Management - Microeconomics


       ----------Course Enrollment Table----------


,0,1,2
0,BUAD 280,BUAD 280 Introduction to Financial Accounting,Emphasis on understanding financial statements...
1,BUAD 281,BUAD 281 Introduction to Managerial Accounting,"An emphasis on how accounting information , co..."
2,ACCT 371,ACCT 371 Introduction to Accounting Systems,Understanding of technology and controls neede...
3,ACCT 478,ACCT 478 Accounting Systems Design,Explores the design of accounting systems . In...
4,ACCT 476,ACCT 476 Performance Measurement Issues,Introduction to understanding how management c...
5,BUAD 302,BUAD 302T Communication Strategy in Accounting,"Theory , practices , and techniques essential ..."
6,BUAD 304,BUAD 304 Organizational Behavior and Leadership,The role of leadership in business organizatio...
7,MATH 118,MATH 118gx Fundamental Principles of Calculus,Derivatives ; extrema . Definite integral ; fu...
8,MATH 125,MATH 125g Calculus I,"Limits ; continuity , derivatives and applicat..."
9,ECON 351,ECON 351x Microeconomics for Business,Development and business applications of : the...


In [66]:
topo_Sort("Accounting (BS)")

    ----------Study Path: School Course with Recommended MOOC----------


Code,Name,Image,MOOC
BUAD 280,BUAD 280 Introduction to Financial Accounting,,Accounting: Principles of Financial Accounting
BUAD 281,BUAD 281 Introduction to Managerial Accounting,,"Managerial Accounting: Cost Behaviors, Systems, and Analysis"
ACCT 371,ACCT 371 Introduction to Accounting Systems,,Accounting and Finance for IT professionals
ACCT 478,ACCT 478 Accounting Systems Design,,More Introduction to Financial Accounting
ACCT 476,ACCT 476 Performance Measurement Issues,,Measurement and data
BUAD 302,BUAD 302T Communication Strategy in Accounting,,
BUAD 304,BUAD 304 Organizational Behavior and Leadership,,International Leadership and Organizational Behavior
MATH 118,MATH 118gx Fundamental Principles of Calculus,,
MATH 125,MATH 125g Calculus I,,Single Variable Calculus
ECON 351,ECON 351x Microeconomics for Business,,Strategic Business Management - Microeconomics


       ----------Course Enrollment Table----------


,0,1,2
0,BUAD 280,BUAD 280 Introduction to Financial Accounting,Emphasis on understanding financial statements...
1,BUAD 281,BUAD 281 Introduction to Managerial Accounting,"An emphasis on how accounting information , co..."
2,ACCT 371,ACCT 371 Introduction to Accounting Systems,Understanding of technology and controls neede...
3,ACCT 478,ACCT 478 Accounting Systems Design,Explores the design of accounting systems . In...
4,ACCT 476,ACCT 476 Performance Measurement Issues,Introduction to understanding how management c...
5,BUAD 302,BUAD 302T Communication Strategy in Accounting,"Theory , practices , and techniques essential ..."
6,BUAD 304,BUAD 304 Organizational Behavior and Leadership,The role of leadership in business organizatio...
7,MATH 118,MATH 118gx Fundamental Principles of Calculus,Derivatives ; extrema . Definite integral ; fu...
8,MATH 125,MATH 125g Calculus I,"Limits ; continuity , derivatives and applicat..."
9,ECON 351,ECON 351x Microeconomics for Business,Development and business applications of : the...
